In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.001
PREDICTION_PATH = "dataset/predictions/02-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
36781,ID_x9OuTG.jpeg,healthy,0.253555,320.663788,1163.428955,1484.858521,1797.367188
22698,ID_xUtJVT.jpg,cssvd,0.000328,0.000000,0.000000,296.327545,200.119705
10518,ID_bl3IBt.jpeg,healthy,0.002112,3352.116943,0.000000,3999.424805,989.017883
15511,ID_F7a5DB.jpg,cssvd,0.018570,5.590732,653.035950,678.489014,1079.270508
2952,ID_LEiRHI.jpg,anthracnose,0.022028,2946.677490,1473.029419,3264.000000,2222.378418


In [4]:
files = list(PREDICTION_IMAGE_PATH.glob("*.*"))
len(files)

1626

In [5]:
print("Size", len(test_df))
test_df: pd.DataFrame = test_df[test_df["confidence"] > LEARNED_THR]
len(test_df)

Size 48780


43280

In [6]:
test_df["Image_ID"].nunique(), len(files)

(1626, 1626)

In [7]:
test_df.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64

In [8]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.5879385471343994,
  'ymin': 196.29763793945312,
  'xmin': 10.367511749267578,
  'ymax': 3934.84912109375,
  'xmax': 1636.239013671875},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.0414691306650638,
  'ymin': 174.48638916015625,
  'xmin': 0.0,
  'ymax': 1687.7427978515625,
  'xmax': 498.4138793945313},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.0302038863301277,
  'ymin': 349.4897766113281,
  'xmin': 1338.6968994140625,
  'ymax': 1136.139892578125,
  'xmax': 1795.4088134765625},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.0201799571514129,
  'ymin': 101.95970153808594,
  'xmin': 1.0755060911178589,
  'ymax': 2287.603271484375,
  'xmax': 567.26904296875},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.0146580608561635,
  'ymin': 0.9587704539299012,
  'xmin': 1253.0130615234375,
  'ymax': 477.12994384765625,
 

In [9]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [10]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)


In [11]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
4819,ID_FiJeqW.jpg,healthy,0.011257,0.000000,1615.074219,738.362732,2445.506348
18269,ID_cCtxAa.jpeg,healthy,0.003804,0.000000,999.107849,227.940567,2470.634521
38608,ID_rGRsVE.jpg,healthy,0.007422,0.009766,0.137520,406.518127,280.764496
33907,ID_KNTwFY.jpg,anthracnose,0.002846,934.409546,903.187012,2044.366333,1466.911377
2174,ID_FZdNUn.jpeg,healthy,0.004395,0.000000,7.574051,277.597900,853.185486
5259,ID_qSCHkr.JPG,healthy,0.008525,0.373499,1883.629761,436.367371,2448.000000
18240,ID_s8kkQp.jpg,healthy,0.004592,216.180374,248.958405,262.313751,292.712494
35152,ID_WBEKHx.jpeg,healthy,0.008058,0.000000,0.161980,242.232620,820.535278
15836,ID_svY6TG.jpg,healthy,0.007355,268.972137,352.766449,405.521271,415.989197
2165,ID_Ho3Skn.jpg,healthy,0.001099,0.428696,0.000000,326.453888,179.607925


In [12]:
test_df["class"].value_counts()

class
healthy        19656
cssvd          12155
anthracnose    11469
Name: count, dtype: int64

In [13]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [14]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/02-submissions.csv'